# Lab-P12:  Web Requests, Caching, DataFrames and Scraping

In this lab, you will practice how to:

* use HTTP requests to download content from the internet,
* cache data onto your computer,
* construct and modify DataFrames to analyze datasets,
* use `BeautifulSoup` to parse web pages and extract useful information.

## Segment 1: Web Requests and File Downloads

We will first import some important modules

In [1]:
# it is considered a good coding practice to place all import statements at the top of the notebook
# please place all your import statements in this cell if you need to import any more modules for this lab

# we have imported these modules for you
import requests
import os
import json
import pandas as pd
from bs4 import BeautifulSoup

### Task 1.1: Fetch `rankings.json` from an internet URL

Use the `requests` library to fetch the file at this URL: `https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/rankings.json`. Make sure to call the appropriate function to **raise** an HTTPError if status code is not `200`.

Then create a variable called `file_text` that saves the text of the response.

**Hint:** You can review the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-29).

In [2]:
# compute and store the answer in the variable 'file_text', but do NOT display it
url = 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/rankings.json'
r = requests.get(url)
file_text = r.text

In [3]:
# run this cell to confirm that your variable has been defined properly

assert file_text[:30] == '[\n    {\n        "World Rank": '

### Task 1.2: Save `rankings.json` as a file

Open a file in write mode called `rankings.json`, and write the contents of the variable `file_text` into it. Make sure to **close** the file afterwards (unless you used a `with` block to open the file).

**Hint:** You can review the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-26)

In [4]:
# write your code here
file = open("rankings.json", "w")
file.write(file_text)
file.close()

In [5]:
# run this cell to confirm that your file has been created

assert(os.path.exists("rankings.json"))

Note that the cell above only checked if the file had been created, and **not** whether it contains the correct data. You must check that yourself. Check your `lab-p12` directory in Finder (Mac) / Explorer (Windows). It should now have a file called `rankings.json`. **Manually open** this file and confirm that it contains the contents of the page [`rankings.json`](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/rankings.json).

### Task 1.3: Implement the `download` function

Now, you will implement a function `download` to download data from the internet and save it to a file. 

This function takes in two arguments `filename` and `url`. The contents at the address pointed to by the `url` field should be saved into the file whose path is specified by `filename`. Remember that you can reuse the code you wrote above.

In [6]:
def download(filename, url):
    # TODO: make the request
    # TODO: raise an HTTPError if status code is not 200
    # TODO: get the text
    # TODO: open the file (with 'utf-8' encoding)
    # TODO: write to the file
    # TODO: close the file
    try:
        r = requests.get(url)
        r.raise_for_status() #similar to asserting r.status_code == 200
        if os.path.exists(filename):
            return (str(filename)+' already exists!')
        file = open(filename, "w")
        file.write(r.text)
        file.close()
    except requests.HTTPError as e: #correct way to catch the error.
        print("oops!!", e)
    return (str(filename) + " created!")

### Task 1.4: Implement caching in the `download` function

The function `download` above is however, not quite what we want. One big disadvantage is that it **downloads** the file even if it has already been created. Fetching data from webpages takes both time and resources, and **must** be avoided as much as possible. In particular, repeatedly downloading files that have been already downloaded is a **very bad** coding practices, and **must** be avoided. 

Your `download` function above is particularly troublesome to use in a Jupyter Notebook, as it will try to fetch data over the internet **every time** your notebook is restarted (or the cell with the call to `download` is run).

Now go back and **modify** `download` to implement *caching*. This means that **before** downloading the file from the internet, the function **must** check if the file already exists. If the file already exists, the function should return the message `"<filename> already exists!"` where `filename` is the argument. It should **not** make a request.

**Hint:** You can use the `os.path.exists` function to check if the `filename` already exists.

### Task 1.5: Test the `download` function

Run the two cells below to test your function. Think about why the test code is written in this way. Ask a TA if you're not sure.

In [28]:
# delete the file if it already exists and download the file

rankings_url = "https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/rankings.json"

if os.path.exists("rankings.json"):
    os.remove("rankings.json") # delete the existing file

assert download("rankings.json", rankings_url) == "rankings.json created!"
assert(os.path.exists("rankings.json"))
assert(os.path.getsize("rankings.json") > 1600000 and os.path.getsize("rankings.json") < 2500000)

In [29]:
# verify that the function does not download again if file already exists

f = open("rankings.json", "w") # rewrite the contents of the file
f.close()
assert (download("rankings.json",rankings_url) == "rankings.json already exists!" )
assert(os.path.getsize("rankings.json") == 0) # check that the file was not redownloaded
os.remove("rankings.json")
assert download("rankings.json", rankings_url) == "rankings.json created!"

You **must** use this `download` function to download files during p12. This will ensure that you do not download the files each time you 'Restart & Run All'.

## Segment 2:  Creating DataFrames

For this project, we will be analyzing statistics about world university rankings adapted from
[here](https://cwur.org/). The `rankings.json` file was created by scraping content from pages on the linked website. 

We are going to use `pandas` throughout the lab and project to analyze this dataset.

### Task 2.1: Load data from `rankings.json` into a dataframe

In lecture, you reviewed different ways to create pandas DataFrames. For this task, create a DataFrame `rankings` by reading the JSON data saved in `rankings.json`. 

In [30]:
# we have done this one for you

rankings = pd.read_json('rankings.json')
rankings.head()

,World Rank,Year,Institution,Country,National Rank,Quality of Education Rank,Alumni Employment Rank,Quality of Faculty Rank,Research Performance Rank,Score
0,1,2019-2020,Harvard University,USA,1,2.0,1.0,1.0,1.0,100.0
1,2,2019-2020,Massachusetts Institute of Technology,USA,2,1.0,10.0,2.0,5.0,96.7
2,3,2019-2020,Stanford University,USA,3,9.0,3.0,3.0,2.0,95.2
3,4,2019-2020,University of Cambridge,United Kingdom,1,4.0,19.0,5.0,11.0,94.1
4,5,2019-2020,University of Oxford,United Kingdom,2,10.0,24.0,10.0,4.0,93.3


In [31]:
# run this cell to test whether 'rankings' has been defined properly

assert(type(rankings) == pd.DataFrame)
assert(rankings.iloc[0]["Institution"] == 'Harvard University')
assert(rankings.iloc[1]["Score"]== 96.7)

### Task 2.2: Find the unique universities in the dataset

As the dataset contains rankings for three different years, the same university may have featured multiple times. Find the names of the unique universities that are represented in the dataset.

First, extract just the names of the institutions as a `pandas` **Series**. Then, make a **list** of unique names called `institutions_list`. **Series** can be easily typecast just like any other data type in Python.

In [32]:
# TODO: create a pandas `Series` of just the institution names in the dataset. 
institutions = rankings.iloc[:]["Institution"]
# TODO: typecast the Series into a set and then into a list, to remove duplicates
institutions_list = list(set(institutions))

In [33]:
# run this cell to test whether 'institutions_list' has been defined properly

assert(type(institutions) == pd.Series)
assert(len(institutions_list) == 2156)

### Task 2.3: Use `value_counts` to count instances in a dataframe

Now, let's find the country that is the 5th most represented in the dataframe, and the number of times it features. Recall that `value_counts` enables us to count number of occurrences of unique values in a pandas **Series**.

#### Task 2.3.1: Obtain the counts for all countries

First, use the `value_counts` function on the `Country` column of `rankings`, and then typecast to a pandas **Series** called `country_counts`. This **Series** should contain each country in the dataset and the number of times it occurs.

**Hint:** You can review the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-27).

In [34]:
# compute and store the answer in the variable 'country_counts', and display it
country_counts = rankings['Country'].value_counts()
country_counts

USA                                 1062
China                                793
Japan                                380
United Kingdom                       284
France                               256
                                    ... 
Democratic Republic of the Congo       1
Palestine                              1
Cuba                                   1
Azerbaijan                             1
Kazakhstan                             1
Name: Country, Length: 103, dtype: int64

In [35]:
# run this cell to test whether 'country_counts' has been defined properly

assert(type(country_counts) == pd.Series)
assert(country_counts["USA"] == 1062)
assert(len(country_counts) == 103)

Note that in the **Series** `country_counts`, the countries are already **sorted** in *decreasing* order of the number of times they appear in `rankings`.

#### Task 2.3.2: Find the 5th most represented country

Use the `.index` attribute of the **Series** `country_counts` to fetch the name of the 5th most represented country. Note that `country_counts` is **sorted** in *decreasing* order of the number of times each country appears in `rankings`. You **must** use `iloc` to fetch the count of this country. Make sure to use the **Series** `country_counts` defined in Task 2.3.1.

**Hint**: The pandas `Series.index` works differently from the `.index` method you are familiar with for **lists**. `Series.index` takes in the numerical **index** of the element you want to access, and returns the **label** you can pass to `.loc` to access it.

In [41]:
# TODO: find the 5th most represented country
fifth_country = country_counts.index[4]

# TODO: find the count of the 5th most represented country
fifth_count = country_counts.iloc[4]


256

In [42]:
# run this cell to test whether 'fifth_country' and 'fifth_count' have been defined properly

assert(fifth_country == "France")
assert(fifth_count == 256)

### Task 2.4: `loc` vs `iloc`

In this lab and project, you **must** only use `iloc`. Using `loc` will be considered **hardcording**. This is because `iloc` selects rows and columns at the given **integer position** while `loc` selects rows at the given **pandas index**. 

Recall that **row index** can be given meaningful names like string indices. Consider a scenario where you add rows to the beginning of the DataFrame - if you use `.loc` indexing, your answer will become **incorrect** if the data changes. Whereas if you use `.iloc`, you will always get the correct answer.

This distinction may not be as intuitive for the current `rankings` **DataFrame**. As an example, use both `loc` and `iloc` to fetch the first row in `rankings`.

In [43]:
# use iloc to extract the first row from 'rankings'

first_row_iloc = rankings.iloc[0]
first_row_iloc

World Rank                                    1
Year                                  2019-2020
Institution                  Harvard University
Country                                     USA
National Rank                                 1
Quality of Education Rank                   2.0
Alumni Employment Rank                      1.0
Quality of Faculty Rank                     1.0
Research Performance Rank                   1.0
Score                                     100.0
Name: 0, dtype: object

In [45]:
# use loc to extract the first row from 'rankings'

first_row_loc = rankings.loc[0]
first_row_loc

World Rank                                    1
Year                                  2019-2020
Institution                  Harvard University
Country                                     USA
National Rank                                 1
Quality of Education Rank                   2.0
Alumni Employment Rank                      1.0
Quality of Faculty Rank                     1.0
Research Performance Rank                   1.0
Score                                     100.0
Name: 0, dtype: object

The results are exactly the same! This happens since the integer positions correspond to the pandas indices in the `rankings` dataframe. However, this will not always hold true - as we see in the next task.

### Task 2.5: Use boolean indexing to filter data

Now, use **boolean indexing** to extract data from the **DataFrame**. You can find the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-28).

Create a **DataFrame** `rankings_arg_bra` that **only** consists of rankings of universities from *Argentina* and *Brazil*. 

**Hints**:When implementing **boolean indexing** in `pandas`, the `or` operator is represented by `|` and the `and` operator is represented by `&`.

In [155]:
# compute and store the answer in the variable 'rankings_arg_bra', then display it
rankings_arg_bra = rankings[(rankings['Country'] == 'Argentina') | (rankings['Country'] == 'Brazil')]
rankings_arg_bra

,World Rank,Year,Institution,Country,National Rank,Quality of Education Rank,Alumni Employment Rank,Quality of Faculty Rank,Research Performance Rank,Score
127,128,2019-2020,University of São Paulo,Brazil,1,457.0,264.0,219.0,89.0,80.7
343,344,2019-2020,University of Buenos Aires,Argentina,1,238.0,1222.0,NaN,320.0,76.1
348,349,2019-2020,Federal University of Rio de Janeiro,Brazil,2,378.0,408.0,NaN,335.0,76.0
352,353,2019-2020,University of Campinas,Brazil,3,NaN,NaN,NaN,324.0,76.0
443,444,2019-2020,São Paulo State University,Brazil,4,NaN,NaN,NaN,416.0,74.8
...,...,...,...,...,...,...,...,...,...,...
5870,1871,2021-2022,Federal Rural University of Rio de Janeiro,Brazil,53,NaN,NaN,NaN,1793.0,66.2
5908,1909,2021-2022,Federal University of Piauí,Brazil,54,NaN,NaN,NaN,1834.0,66.1
5944,1945,2021-2022,Federal University of Amazonas,Brazil,55,NaN,NaN,NaN,1870.0,65.9
5975,1976,2021-2022,National University of Tucumán,Argentina,10,NaN,NaN,NaN,1901.0,65.8


Now, we will try to extract the **first** value in this new **DataFrame** using `iloc` and `loc`. As you'll see, using `loc` will not work the same way it did before. In fact, it will throw an **error**. To verify, run the two cells below:

In [156]:
first_row_iloc = rankings_arg_bra.iloc[0]
first_row_iloc

World Rank                                       128
Year                                       2019-2020
Institution                  University of São Paulo
Country                                       Brazil
National Rank                                      1
Quality of Education Rank                      457.0
Alumni Employment Rank                         264.0
Quality of Faculty Rank                        219.0
Research Performance Rank                       89.0
Score                                           80.7
Name: 127, dtype: object

In [157]:
#first_row_loc = rankings_arg_bra.loc[0]
#first_row_loc 

We see that using `.loc` now causes a **KeyError**.

`.loc[0]` tries to find the row with the *labeled* **index** 0. Run the cell below and notice how `rankings_arg_bra` starts at the labeled **index** 127. There is no 0. Hence the **KeyError**.

In [158]:
rankings_arg_bra.head()

,World Rank,Year,Institution,Country,National Rank,Quality of Education Rank,Alumni Employment Rank,Quality of Faculty Rank,Research Performance Rank,Score
127,128,2019-2020,University of São Paulo,Brazil,1,457.0,264.0,219.0,89.0,80.7
343,344,2019-2020,University of Buenos Aires,Argentina,1,238.0,1222.0,NaN,320.0,76.1
348,349,2019-2020,Federal University of Rio de Janeiro,Brazil,2,378.0,408.0,NaN,335.0,76.0
352,353,2019-2020,University of Campinas,Brazil,3,NaN,NaN,NaN,324.0,76.0
443,444,2019-2020,São Paulo State University,Brazil,4,NaN,NaN,NaN,416.0,74.8


### Task 2.6: Sort the DataFrame

The **DataFrame** in Task 2.5 is sorted by `World Rank`, with the result that universities from *Argentina* and *Brazil* are interleaved throughout the data. **Re-sort** the data to sort by `Country` so that all universities from *Argentina* appear **first** followed by universities from *Brazil*. Within each country, the universities should be **sorted** by their `National Rank`. 

Use the `sort_values` function of `pandas`. You can find the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-28). Remember - by default, `pandas` returns a **new** sorted **DataFrame** and does **not** modify the existing one.

Recall that `sort_values` takes an argument for the parameter `by` as the column name, based on which you want to do the sorting. If you want to use one column for primary sorting and another for secondary sorting, you can specify a **list** of column names.

In [57]:
# compute and store the answer in the variable 'sorted_rankings_arg_bra', then display it
sorted_rankings_arg_bra = rankings_arg_bra.sort_values(by=['Country','National Rank'])
sorted_rankings_arg_bra

,World Rank,Year,Institution,Country,National Rank,Quality of Education Rank,Alumni Employment Rank,Quality of Faculty Rank,Research Performance Rank,Score
343,344,2019-2020,University of Buenos Aires,Argentina,1,238.0,1222.0,NaN,320.0,76.1
2353,354,2020-2021,University of Buenos Aires,Argentina,1,327.0,1281.0,NaN,321.0,75.9
4355,356,2021-2022,University of Buenos Aires,Argentina,1,319.0,1347.0,NaN,324.0,75.9
595,596,2019-2020,National University of La Plata,Argentina,2,495.0,1337.0,NaN,557.0,73.2
2618,619,2020-2021,National University of La Plata,Argentina,2,NaN,1404.0,NaN,583.0,73.0
...,...,...,...,...,...,...,...,...,...,...
3947,1948,2020-2021,Federal University of Amazonas,Brazil,55,NaN,NaN,NaN,1870.0,66.0
5944,1945,2021-2022,Federal University of Amazonas,Brazil,55,NaN,NaN,NaN,1870.0,65.9
3991,1992,2020-2021,Santa Catarina State University,Brazil,56,NaN,1019.0,NaN,1916.0,65.8
5995,1996,2021-2022,Santa Catarina State University,Brazil,56,NaN,936.0,NaN,1938.0,65.8


In [58]:
# run this cell to test whether 'sorted_rankings_arg_bra' has been defined properly

assert(sorted_rankings_arg_bra.iloc[0]["Institution"] == "University of Buenos Aires")
assert(sorted_rankings_arg_bra.iloc[-1]["World Rank"] == 1997)

### Task 2.7: Create a new, simplified DataFrame to track changes in rankings

As we have seen, universities that have featured in rankings of multiple years are featured repeatedly. To simplify comparisons, we want to feature each university once and remove all other metrics. 

This time - instead of simply ranking universities, we want to find the **absolute change** in universities' rankings between the years *2019-2020* and *2020-2021*. We are only interested in the absolute change and **not** whether the rank improved or declined.  

#### Task 2.7.1: Find the absolute difference in World Rank for "University of Madras" between 2019-2020 and 2020-2021

First, let's attempt to measure the change for one particular university.

In [101]:
# compute and store the answer in the variable 'absolute_diff_madras', then display it

# TODO: first find the ranking of "University of Madras" in the year "2019-2020"
# TODO: then find the ranking of "University of Madras" in the year "2020-2021
# TODO: Remember to use .iloc[0] to extract the value
um_1920 = rankings[(rankings['Institution'] == "University of Madras") & (rankings['Year'] == "2019-2020")].iloc[0]['World Rank']
um_2021 = rankings[(rankings['Institution'] == "University of Madras") & (rankings['Year'] == "2020-2021")].iloc[0]['World Rank']
absolute_diff_madras = abs(um_1920 - um_2021)

In [102]:
# run this cell to test whether 'absolute_diff_madras' has been defined properly

assert(absolute_diff_madras == 108)

#### Task 2.7.2: Create a Series with the absolute difference in ranks for "University of Madras" between 2019-2020 and 2020-2021

Now, create a **dictionary** with the keys as `Institution` and `Absolute Change`. The values should be the relevant values for *University of Madras*. Then, convert this **dictionary** to a **Series**.

In [103]:
# compute and store the answer in the variable 'madras_series', then display it

# TODO: first define the dictionary 'madras_dict'
madras_dict = {
    'Institution':"University of Madras",
    'Absolute Change':absolute_diff_madras
}
# TODO: then you can convert to a Series
madras_series = pd.Series(madras_dict)

madras_series

Institution        University of Madras
Absolute Change                     108
dtype: object

In [104]:
# run this cell to test whether 'madras_series' has been defined properly

assert(madras_series["Institution"] == "University of Madras")
assert(madras_series["Absolute Change"] == 108)

#### Task 2.7.3: Create the `change_in_rankings` DataFrame

Now, create a **DataFrame** `change_in_rankings` with just 2 columns, `Institution` and `Absolute Change` where **each** university is only featured once. If the institution is **not** present in the rankings of **both** years, we will just ignore it.

The institutions should be **sorted** in *increasing* order of their **absolute change**. For institutions with the **same** absolute change, sort them *alphabetically* by their **names**.

**Warning:** Even if your code is optimal, this cell may take a few seconds to run. However, if it takes much longer than that (say, if it takes 30 seconds or longer), then you will **need** to optimize your code so it runs faster.

In [159]:
# suggested approach (you may do it some other way if you wish)

# TODO: initialize an empty list
# TODO: create a list of unique institutions in the DataFrame 'rankings'
# TODO: loop through these institutions
    # TODO: create a new dataframe that has rankings for only this institution
    #       (hint: Use boolean indexing for the "Institution" column)
    # TODO: create a list of years by casting the "Year" column of this dataframe to a list
    # TODO: skip institution if "2019-2020" or "2020-2021" are *not* in this list
        
    # TODO: extract the "World Rank" for both years from the new DataFrame 
    #       (remember to use .iloc[0] to extract the actual value)
    # TODO: find their absolute difference
       
    # TODO: make a dictionary where the keys are “Institution” and “Absolute Change”
    #       and the values are the corresponding values you just found for this institution
    
    # TODO: append this dictionary to the list initialized in the first step

# TODO: finally, convert the list of dicts to a pandas DataFrame called change_in_rankings
# TODO: sort this DataFrame using sort_values
change_in_rankings_dict = {}
ins_unique = list(set(rankings['Institution'].iloc[:].tolist()))
list_1920 = rankings[(rankings['Year'] == '2019-2020')]
list_2021 = rankings[(rankings['Year'] == '2020-2021')]
for ins in ins_unique:
    if ins not in list_1920['Institution'].tolist() or ins not in list_2021['Institution'].tolist():
        continue
    ins_1920 = rankings[(rankings['Institution'] == ins) & (rankings['Year'] == "2019-2020")].iloc[0]['World Rank']
    ins_2021 = rankings[(rankings['Institution'] == ins) & (rankings['Year'] == "2020-2021")].iloc[0]['World Rank']
    absolute_diff = abs(ins_1920 - ins_2021)
    change_in_rankings_dict[ins] = absolute_diff
change_in_rankings = pd.DataFrame(change_in_rankings_dict.items(), columns=['Institution', 'Absolute Change']).sort_values(by = ['Absolute Change','Institution'])

In [160]:
# run this cell to test whether 'change_in_rankings' has been defined properly

assert(change_in_rankings.iloc[100]["Institution"] == "Vrije Universiteit Brussel")
assert(change_in_rankings.iloc[-1]["Absolute Change"] == 1081)
assert(change_in_rankings.shape[1] == 2)

# Segment 3: BeautifulSoup

As mentioned in Segment 2, the `rankings.json` file was created by parsing HTML content on the Web, specifically the web pages listed below.

* https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2019-2020.html
* https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2020-2021.html
* https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2021-2022.html

Now, let's write a function to do this ourselves. We will use the `BeautifulSoup` module to scrape web pages and extract information. It is recommended that you review the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-31) if you get stuck at any point in this segment.

### Task 3.1: Download the HTML files

Use the `download` function you previously created to download the contents of each of the URLs above and save them into files. Name the files `2019-2020.html`, `2020-2021.html` and `2021-2022.html` based on the respective URL.

In [165]:
# use the 'download' function to download the data from the webpage
# 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2019-2020.html'
# to the file '2019-2020.html'
file_name1 = '2019-2020.html'
url1 = 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2019-2020.html'
download(file_name1, url1)

'2019-2020.html already exists!'

In [166]:
# use the 'download' function to download the data from the webpage
# 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2020-2021.html'
# to the file '2020-2021.html'
file_name2 = '2020-2021.html'
url2 = 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2020-2021.html'
download(file_name2, url2)

'2020-2021.html already exists!'

In [167]:
# use the 'download' function to download the data from the webpage
# 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2021-2022.html'
# to the file '2021-2022.html'
file_name3 = '2021-2022.html'
url3 = 'https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-f22-projects/-/raw/main/p12/2021-2022.html'
download(file_name3, url3)

'2021-2022.html already exists!'

### Task 3.2: Read `2019-2020.html`

First, read the contents of the file `2019-2020.html`.

**Hint:** If you get a `UnicodeDecodeError`, make sure all your calls to `open` have the keyword argument `encoding="utf-8"`.

In [216]:
# write your code here - you may use any variable names you want to
#r = requests.get(url1)
#contents_1= r.text

f = open("2019-2020.html")
contents_1 = f.read()
f.close()

### Task 3.3: Initialize `BeautifulSoup` object instance

Use the variable defined in Task 3.2. You can review the relevant lecture code [here](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f22/meena_lec_notes/lec-31).

In [217]:
# write your code here - you may use any variable names you want to
bs_obj = BeautifulSoup(contents_1, "html.parser") 

### Task 3.4: Find the `table` element

The webpage has a `table` containing all the data we're trying to extract. Write the code to **find** this element and store it in a variable. Use the `BeautifulSoup` object instance defined in Task 3.3.

In [199]:
# write your code here - you may use any variable names you want to
table = bs_obj.find("table") 


### Task 3.5: Find all `th` tags, to parse the table header

Use the variable defined in Task 3.4.

**Hint**: The **header** should be a **list** of elements, that can be obtained by using the `get_text` method for each `th` element in the table. You may also find list comprehension useful here.

In [200]:
# compute and store the answer in the variable 'header', then display it
#header = bs_obj.find("th") 
header = [th.get_text() for th in table.find_all('th')]
header

['World Rank',
 'Institution',
 'Country',
 'National Rank',
 'Quality of Education Rank',
 'Alumni Employment Rank',
 'Quality of Faculty Rank',
 'Research Performance Rank',
 'Score']

In [201]:
# run this cell to test whether 'header' has been defined properly

assert(len(header) == 9)
assert(type(header) == list)
assert(header[0] == "World Rank")
assert(header[-1] == "Score")

### Task 3.6: Build row dictionary for one row

Scrape the **second** row (the first one is the **header**!) of the `table`, convert the data to the appropriate **data types**, and populate the data into a **dictionary**. The **keys** of the dictionary **must** be the columns in the **DataFrame**. **Avoid** *hardcoding* these **keys** - instead, use the variable `header` obtained in the previous task.

**Hint**: Rows can be found by locating the `tr` elements in the table. After identifying the second row of the table, try to figure out which tag separates the different columns.

The required data types for each column is:

|**Column Name**|**Data Type**|
|---------------|-------------|
|`World Rank`|**int**|
|`Institution`|**str**|
|`Country`|**str**|
|`National Rank`|**int**|
|`Quality of Education Rank`|**int**|
|`Alumni Employment Rank`|**int**|
|`Quality of Faculty Rank`|**int**|
|`Research Performance Rank`|**int**|
|`Score`|**float**|

You can **compare** your output with the data in `rankings.json`, to confirm whether you have parsed the file correctly (note that you do **not** yet have to implement the `Year` column in your **dicitonary**).

In [202]:
# write your code here - you may use any variable names you want to

rank = {}

tr_elements = table.find_all('tr')
tr = tr_elements[1]
td_elements = tr.find_all('td')
for idx in range(len(td_elements)):
    td = td_elements[idx]
    val = td.get_text()
    if header[idx] in ["World Rank", "National Rank", 'Quality of Education Rank', 'Alumni Employment Rank', 'Quality of Faculty Rank', 'Research Performance Rank']:
        rank[header[idx]] = int(val)
    elif header[idx] in ['Institution', 'Country']:
        rank[header[idx]] = str(val)
    elif header[idx] == "Score":
        rank[header[idx]] = float(val)
    
rank

{'World Rank': 1,
 'Institution': 'Harvard University',
 'Country': 'USA',
 'National Rank': 1,
 'Quality of Education Rank': 2,
 'Alumni Employment Rank': 1,
 'Quality of Faculty Rank': 1,
 'Research Performance Rank': 1,
 'Score': 100.0}

### Task 3.7: Build list of all row dictionaries

Scrape **all** rows, **convert** data to appropriate types, and populate data into a row **dictionary** and append row all dictionaries to a **list**.

This is a natural extension of Task 3.6. You can use a loop to extract all rows and populate the list.

**Important**:
* Some fields in the dataset have **missing** values, represented simply as `"-"`. Such **missing** values should be replaced by `None` in your **dictionary**.

In [208]:
# write your code here - you may use any variable names you want to

rank_data = []

tr_elements = table.find_all('tr')
for tr in tr_elements[1:]: # Skip first row (header row)
    rank = {}
    td_elements = tr.find_all('td')
    for idx in range(len(td_elements)):
        td = td_elements[idx]
        val = td.get_text()
        if header[idx] in ["World Rank", "National Rank", 'Quality of Education Rank', 'Alumni Employment Rank', 'Quality of Faculty Rank', 'Research Performance Rank']:
            if val == '-':
                rank[header[idx]] = None
            else:
                rank[header[idx]] = int(val)
        elif header[idx] in ['Institution', 'Country']:
            rank[header[idx]] = str(val)
        elif header[idx] == "Score":
            rank[header[idx]] = float(val)
    rank_data.append(rank)
rank_data

[{'World Rank': 1,
  'Institution': 'Harvard University',
  'Country': 'USA',
  'National Rank': 1,
  'Quality of Education Rank': 2,
  'Alumni Employment Rank': 1,
  'Quality of Faculty Rank': 1,
  'Research Performance Rank': 1,
  'Score': 100.0},
 {'World Rank': 2,
  'Institution': 'Massachusetts Institute of Technology',
  'Country': 'USA',
  'National Rank': 2,
  'Quality of Education Rank': 1,
  'Alumni Employment Rank': 10,
  'Quality of Faculty Rank': 2,
  'Research Performance Rank': 5,
  'Score': 96.7},
 {'World Rank': 3,
  'Institution': 'Stanford University',
  'Country': 'USA',
  'National Rank': 3,
  'Quality of Education Rank': 9,
  'Alumni Employment Rank': 3,
  'Quality of Faculty Rank': 3,
  'Research Performance Rank': 2,
  'Score': 95.2},
 {'World Rank': 4,
  'Institution': 'University of Cambridge',
  'Country': 'United Kingdom',
  'National Rank': 1,
  'Quality of Education Rank': 4,
  'Alumni Employment Rank': 19,
  'Quality of Faculty Rank': 5,
  'Research Perfo

### Task 3.8: Write the `parse_html` function

Convert tasks 3.2 to 3.7 to a **function**. This function `parse_html` should take in a `filename` as **input** and **return** a **list** of **dictionaries**, with each **dictionary** representing a **row** in the dataset.

Additionally, we **also** want to include the **key** `Year` to all our **dictionaries**. The `Year` value is **not** present in the dataset. You can extract this value from the `filename`.

In [226]:
def parse_html(filename):
    '''parse_html(filename) parses an HTML file and 
    returns a list of dictionaries containing the tabular data'''
    f = open(filename)
    contents_1 = f.read()
    f.close()
    bs_obj = BeautifulSoup(contents_1, "html.parser") 
    table = bs_obj.find("table") 
    header = [th.get_text() for th in table.find_all('th')]
    rank_data = []
    tr_elements = table.find_all('tr')
    for tr in tr_elements[1:]: 
        rank = {}
        td_elements = tr.find_all('td')
        for idx in range(len(td_elements)):
            td = td_elements[idx]
            val = td.get_text()
            if header[idx] in ["World Rank", "National Rank", 'Quality of Education Rank', 'Alumni Employment Rank', 'Quality of Faculty Rank', 'Research Performance Rank']:
                if val == '-':
                    rank[header[idx]] = None
                else:
                    rank[header[idx]] = int(val)
            elif header[idx] in ['Institution', 'Country']:
                rank[header[idx]] = str(val)
            elif header[idx] == "Score":
                rank[header[idx]] = float(val)
        rank['Year'] = str(filename[0:9])
        rank_data.append(rank)
    return rank_data

In [227]:
# run this cell to test whether 'parse_html' has been defined properly

parsed_2019_2020 = parse_html("2019-2020.html")
parsed_2020_2021 = parse_html("2020-2021.html")
parsed_2021_2022 = parse_html("2021-2022.html")
assert(parsed_2019_2020[-1]["Institution"] == 'Government College University Faisalabad')
assert(parsed_2020_2021[15]["Score"] == 89.0)
assert(parsed_2021_2022[100]["Country"] == 'United Kingdom')
assert(parsed_2021_2022[25]["World Rank"] == 26)
assert(parsed_2020_2021[-5]["National Rank"] == 15)
assert(parsed_2019_2020[50]["Quality of Faculty Rank"] == 78)
assert(parsed_2021_2022[87]["Alumni Employment Rank"] == 464)
assert(parsed_2020_2021[40]["Research Performance Rank"] == 398)
assert(parsed_2019_2020[0]["Year"] == "2019-2020")

If you fail any of the checks above, you can open `rankings.json` **manually** and compare with the output of your `parse_html` function.

### Congratulations, you are now ready to start p12!